# Titanic Simply understand Best Model



### **Introduction**
This is my first stab at a Kaggle script. I have chosen to work with the Titanic dataset after spending some time poking around on the site and looking at other scripts made by other Kagglers for inspiration. 

We will have a detailed statistical analysis of Titanic data set along with Machine learning model implementation.

<h2 style="color:blue"><center> Don't forget to upvote📈 if you like👍🏻 it! It's free! 🎗️🎯 

## Table of content:
* About Titanic
* Import Necessary Libraries
* Handle Missing Values
* Data Exploration/ Analysis/ Visualizing
* Correlation & Correlation Matrix
* Feature Engineering
* Predictive Modeling 
>  1. Logistic Regression
>  2. KNN Classifier
>  3. Gaussian Naive Bayes
>  4. Support Vector Machine(SVM)
>  5. Decision Tree
>  6. Random Forest
* Confusion Matrix
* Precision and Recall
* F-Score
* ROC AUC Curve


### About Titanic

The RMS Titanic was a British passenger liner that sank in the North Atlantic Ocean in the early morning hours of 15 April 1912, after it collided with an iceberg during its maiden voyage from Southampton to New York City. There were an estimated 2,224 passengers and crew aboard the ship, and more than 1,500 passengers and crew died, That's why the name DieTanic. The Titanic was the largest ship afloat at the time it entered service and was the second of three Olympic-class ocean liners operated by the White Star Line. The Titanic was built by the Harland and Wolff shipyard in Belfast. Thomas Andrews, her architect, died in the unforgettable disaster.

<img src="https://i.ibb.co/7jqRQ7c/Titanic-side-plan.png" alt="Titanic" style="width:850px;height:300px;">
<ul>

<br>

## Import Necessary Libraries and Data Sets.
-----------------------------

In [ ]:
# data processing
import pandas as pd

## linear algebra
import numpy as np

# data visualization
import seaborn as sns
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import style

# Algorithms
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB
 
from sklearn.metrics import accuracy_score  #for accuracy_score
from sklearn.model_selection import KFold #for K-fold cross validation
from sklearn.model_selection import cross_val_score #score evaluation
from sklearn.model_selection import cross_val_predict #prediction
from sklearn.metrics import confusion_matrix #for confusion matrix

## Getting  the data
----------------

In [ ]:
titanic = pd.read_csv('../input/titanic/train.csv')
# Print the first 5 rows of the dataframe.
titanic.head(5)


In [ ]:
titanic_test = pd.read_csv('../input/titanic/test.csv')
# Print the last 5 rows of the dataframe.
titanic_test.tail(5)

Note: There is no  survived column in test data, which is our target variable and we will going to predict it.

## Data Exploration/Analysis
--------

In [ ]:
#shape command will give number of rows/samples/examples and number of columns/features/predictors in dataset
#(rows,columns)
titanic.shape

In [ ]:
# Describe gives statistical information about numerical columns in the dataset
titanic.describe()
#you can check from count if there are missing values in columns, here we can see there are some missing values in column "Age"

In [ ]:
# info method provides information about dataset like.
# total values in each column, null/not null, datatype, memory occupied etc.
titanic.info()

## Lets check missing data
----

In [ ]:
# Let's write a function to print the total percentage of the missing values.
# (this can be a good exercise for beginners to try to write simple functions like this.)

#This function takes a DataFrame(df) as input and returns two columns, total missing values and total missing values percentage
def missing_data(df):
    total = df.isnull().sum().sort_values(ascending = False)
    percent = round(df.isnull().sum().sort_values(ascending = False) * 100 /len(df),2)
    return pd.concat([total, percent], axis=1, keys=['Total','Percent'])

In [ ]:
# lets check missing values by columns
missing_data(titanic)

We can see that Age, Embarked and cabin has missing values.
now, lets check missing values for test data.

In [ ]:
# Check missing values in test data set
missing_data(titanic_test)

Age, Fare and cabin has missing values. we will see how to work with these missing values next.
#### Column "Cabin" has more than 75% of missing values in Train and test both dataset, so we are remove it.  
#### Suggestion: Not to impute missing data in columns, which have more than 40% of missing data. 

In [ ]:
drop_column = ['Cabin']
titanic.drop(drop_column, axis= 1, inplace = True)
titanic_test.drop(drop_column,axis = 1,inplace = True)

In [ ]:
#COMPLETING: complete or delete missing values in train and test/validation dataset
dataset = [titanic, titanic_test]

# def missing_data(x):
for data in dataset:
    #complete missing age with median
    data['Age'].fillna(data['Age'].median(), inplace = True)

    #complete missing Embarked with Mode
    data['Embarked'].fillna(data['Embarked'].mode()[0], inplace = True)

    #complete missing Fare with median
    data['Fare'].fillna(data['Fare'].median(), inplace = True)
      
missing_data(titanic)

## Data Exploration/ Analysis/ Visualizing 
------
#### Let's do analysis of data and Check which features could contribute to a high survival rate ?

### 1.  Survived Count
Lets visualize how many passenger & crew survived and how many not. for that create frequency diagram or count plot. 

In [ ]:
def draw(graph):
    for p in graph.patches:
        height = p.get_height()
        graph.text(p.get_x()+p.get_width()/2., height + 5,height ,ha= "center")

In [ ]:
sns.set(style="darkgrid")
plt.figure(figsize = (8, 5))
graph= sns.countplot(x='Survived', hue="Survived", data=titanic)
draw(graph)

### 2. Sex vs Survived
Lets check How many male and female survival.

In [ ]:
plt.figure(figsize = (8, 5))
graph  = sns.countplot(x ="Sex", hue ="Survived", data = titanic)
draw(graph)

### 3. Embarked & PClass vs Survived

As per the graph we can say that Gender is matter to survival if gender is Female then higher survival chances.
Now we will check from which location(Embarked & PClass) gender have more chances to survive.

* **Embarked:** From which location passenger go on board to Titanic.
 * C = Cherbourg
 * Q = Queenstown 
 * S = Southampton
 
* **PClass:** Passenger belongs to which class.
 * 1st = Upper
 * 2nd = Middle
 * 3rd = Lower

First Lets check how many person in different Embarked and Passenger Class. 

In [ ]:
 
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10,4)) 
x = sns.countplot(titanic['Pclass'], ax=ax[0])
y = sns.countplot(titanic['Embarked'], ax=ax[1])
draw(x)
draw(y)
fig.show()

In [ ]:
FacetGrid = sns.FacetGrid(titanic, col='Pclass', height=4, aspect=1)
FacetGrid.map(sns.pointplot, 'Embarked', 'Survived', 'Sex', palette=None,  order=None, hue_order=None)
FacetGrid.add_legend()

What do you think Embarked is important feature to predict high survival rate?  

**From which location passenger go on board to Titanic does it matter or its more important that passenger is on Titanic, no matter from where you go on board to Titanic.** as we know that At **2:20 a.m. on April 15, 1912**, the British ocean liner Titanic sinks into the North Atlantic Ocean.

Its night time, high cold weather (The temperature of the water was -2.2 degrees Celsius when Titanic was sinking), and Job-Location/Rest-room(Passenger class) allocated to everyone on Titanic.

We can use **Embarked** as feature here for getting high accuracy but logically its doesn't matter. so we drop it out.

As a part of data science you have to thing 360 degree angle, some features are important but in calculation(mathematically) its not, so that why you must have domain knowledge for feature selection.

In [ ]:
drop_column = ['Embarked']
titanic.drop(drop_column, axis=1, inplace = True)
titanic_test.drop(drop_column,axis=1,inplace=True)

### 4. Passenger Class Vs Survived
Lets check Pclass impact on survival, as we already know the location of different passenger class.

* 1st = Upper
* 2nd = Middle
* 3rd = Lower

In [ ]:
plt.figure(figsize = (8, 5))
pclass= sns.countplot(x='Pclass', hue='Survived', data=titanic)
draw(pclass)

Here we see clearly, that Pclass is contributing to a persons chance of survival, especially if this person is in class 1. We will create another pclass plot below.

In [ ]:
plt.figure(figsize = (8, 5))
sns.barplot(x='Pclass', y='Survived', data=titanic)

### 5. SibSp & Parch vs Survived

**SibSp** and **Parch** would make more sense, Parents not let child die, Bond of Blood relation always help each other first, rather than helping others they think about them self and them family member. 

Create new feature Family Size as a combination of SibSp and Parch,

In [ ]:
# combine test and train as single to apply some function, we will use it again in Data Preprocessing
all_data=[titanic,titanic_test]

for dataset in all_data:
    dataset['Family'] = dataset['SibSp'] + dataset['Parch'] + 1

In [ ]:
axes = sns.factorplot('Family','Survived', 
                      data=titanic, aspect = 2.5, )

Graph of survival vs family represent that 

- If you are alone then your survival chances is around 25%
- If you have family of 4 members then survival chances is highest around 70%
- If you have more family members then your survival chances is reducing. 

### 6. Age vs Survived
Here another feature is important the age value & sex of all family members.

But as we already seen that for sex Female have more probability of survive as compare to Male.

So lets check Combination of Family and Age for survival.

In [ ]:
axes = sns.factorplot('Family','Age','Survived',
                      data=titanic, aspect = 2.5, )

Now you get more confident that person who have age around 30 have more chances to survive.

And in family if all members age near to 30 then survival probability is very high. But in real life(practically) very less chances. If all young siblings travel in Titanic then its work here.

But we can see that if you are alone and your age is around 30 then your survival chances are 50%

While you with family with 5 members and all are around 30 then your survival chance very high.

So family & age features are very important.

In [ ]:
# create bin for age features. 
for dataset in all_data:
    dataset['Age_bin'] = pd.cut(dataset['Age'], bins=[0,12,20,40,120], labels=['Children','Teenage','Adult','Elder'])
    
plt.figure(figsize = (8, 5))
sns.barplot(x='Age_bin', y='Survived', data=titanic)

Here is only age no relation with family passenger is calculated alone with his/her age.

So children who have age less than 12 years have high chances to survival.

now think whats logic behind that?

Yes, Parents and elder siblings. who think to save them younger once first. 

But here display probability/chances to survival only not displayed count how many Children, Teenager, Adults and Elders on Titanic. So lets check Count of survived for each age.

In [ ]:
plt.figure(figsize = (8, 5))
ag = sns.countplot(x='Age_bin', hue='Survived', data=titanic)
draw(ag)

Here we see age is important feature so lets check relation of age and sex for survived.

This different combination give us idea for selection of correct feature by programmatically and logically.

So lets plot the graph of Age & Sex vs Survived

In [ ]:
AAS = titanic[['Sex','Age_bin','Survived']].groupby(['Sex','Age_bin'],as_index=False).mean()
sns.factorplot('Age_bin','Survived','Sex', data=AAS
                ,aspect=3,kind='bar')
plt.suptitle('Age , Sex vs Survived')

Looks like very unfair with male passenger,

in all age band Female have more chances to survived.

**Think on this:** Male are good person to help them or Female are smart person to survive. 

### 7. Fare Vs Survived

Before we go ahead lets understand the feature "Fare"

**What is "Fare"?**

A fare is the fee paid by a passenger for travel in Titanic in other words we can say Fare is the Ticket price.

Lets check Fare value is related to Passenger class? Logically YES.

so, lets plot graph of it.

In [ ]:
# create bin for fare features
for dataset in all_data:
    dataset['Fare_bin'] = pd.cut(dataset['Fare'], bins=[0,10,50,100,550], labels=['Low_fare','median_fare','Average_fare','high_fare'])
plt.figure(figsize = (8, 5))
ag = sns.countplot(x='Pclass', hue='Fare_bin', data=titanic)

So we can say that fare is correlated with Passenger class.

High fare then upper class, Low fare then lower class.

Now check For survived relation with Fare_bin and compare result of it with Pclass vs Survived.

In [ ]:
sns.barplot(x='Fare_bin', y='Survived', data=titanic)

High fare for passenger class one, and survived count are higher for Pclass-1 and High fare. 

Low fare for passenger class three, and survived count are lower for Pclass-3 and low fare.

so they are co-related with each other. 

Now we drop fare form our data set but before that we check correlation.

## Correlation & Correlation Matrix
----

The first thing to note is that only the numeric features are compared as it is obvious that we cannot correlate between alphabets or strings. Before understanding the plot, let us see what exactly correlation is.

**POSITIVE CORRELATION:** If an increase in feature A leads to increase in feature B, then they are positively correlated. A value 1 means perfect positive correlation.

**NEGATIVE CORRELATION:** If an increase in feature A leads to decrease in feature B, then they are negatively correlated. A value -1 means perfect negative correlation.

In [ ]:
pd.DataFrame(abs(titanic.corr()['Survived']).sort_values(ascending = False))

In [ ]:
# Generate a mask for the upper triangle (taken from seaborn example gallery)
corr=titanic.corr()#['Survived']

mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
plt.subplots(figsize = (12,8))
sns.heatmap(corr, 
            annot=True,
            mask = mask,
            cmap = 'RdBu',
            linewidths=.9, 
            linecolor='white',
            vmax = 0.3,
            fmt='.2f',
            center = 0,
            square=True)
plt.title("Correlations Matrix", y = 1,fontsize = 20, pad = 20);

### Feature engineering

Feature engineering is the art of converting raw data into useful features. There are several feature engineering techniques that you can apply to be an artist.

In [ ]:
titanic.info()

In [ ]:
# Convert ‘Sex’ feature into numeric.
genders = {"male": 0, "female": 1}

for dataset in all_data:
    dataset['Sex'] = dataset['Sex'].map(genders)
titanic['Sex'].value_counts()

In [ ]:
for dataset in all_data:
    dataset['Age'] = dataset['Age'].astype(int)
    dataset.loc[ dataset['Age'] <= 15, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 15) & (dataset['Age'] <= 20), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 20) & (dataset['Age'] <= 26), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 26) & (dataset['Age'] <= 28), 'Age'] = 3
    dataset.loc[(dataset['Age'] > 28) & (dataset['Age'] <= 35), 'Age'] = 4
    dataset.loc[(dataset['Age'] > 35) & (dataset['Age'] <= 45), 'Age'] = 5
    dataset.loc[ dataset['Age'] > 45, 'Age'] = 6
titanic['Age'].value_counts()

In [ ]:
# As we created new fetures form existing one, so we remove that one.

# Removing SibSp & Parch because we have family now. same way Age.
# We also going to remove some other features like passenger id in list, Ticket number and Name.

for dataset in all_data:
    drop_column = ['Age_bin','Fare','Name','Ticket', 'PassengerId','SibSp','Parch','Fare_bin']
    dataset.drop(drop_column, axis=1, inplace = True)


## Predictive Modeling
We have gained some insights from the EDA part. But with that, we cannot accurately predict or tell whether a passenger will survive or die. So now we will predict the whether the Passenger will survive or not using some great Classification Algorithms.

In [ ]:
all_features = titanic.drop("Survived",axis=1)
Targete = titanic["Survived"]
X_train,X_test,y_train,y_test = train_test_split(all_features,Targete,test_size=0.3,random_state=0)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

### 1. Logistic Regression:

In [ ]:
model = LogisticRegression()
model.fit(X_train,y_train)
prediction_lr=model.predict(X_test)
Log_acc = round(accuracy_score(prediction_lr,y_test)*100,2)
kfold = KFold(n_splits=10, random_state=22) # k=10, split the data into 10 equal parts
Log_cv_acc=cross_val_score(model,all_features,Targete,cv=10,scoring='accuracy')

print('The accuracy of the Logistic Regression is',Log_acc)
print('The cross validated score for Logistic REgression is:',round(Log_cv_acc.mean()*100,2))

### 2. K Nearest Neighbor:

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 3) 
knn.fit(X_train, y_train)  
Y_pred = knn.predict(X_test)  
acc_knn = round(knn.score(X_train, y_train) * 100, 2)
kfold = KFold(n_splits=10, random_state=22) 
result_knn=cross_val_score(model,all_features,Targete,cv=10,scoring='accuracy')

print('The accuracy of the K Nearst Neighbors Classifier is',round(accuracy_score(Y_pred,y_test)*100,2))
print('The cross validated score for K Nearest Neighbors Classifier is:',round(result_knn.mean()*100,2))

### 3. Gaussian Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
model= GaussianNB()
model.fit(X_train,y_train)
prediction_gnb=model.predict(X_test) 
nb_acc = round(accuracy_score(prediction_gnb,y_test)*100,2)
kfold = KFold(n_splits=12, random_state=22)
result_gnb=cross_val_score(model,all_features,Targete,cv=12,scoring='accuracy')

print('The accuracy of the Gaussian Naive Bayes Classifier is',nb_acc)
print('The cross validated score for Gaussian Naive Bayes classifier is:',round(result_gnb.mean()*100,2))

### 4. Linear Support Vector Machine:

In [ ]:
linear_svc = LinearSVC()
linear_svc.fit(X_train, y_train)

Y_pred = linear_svc.predict(X_test)

acc_linear_svc = round(linear_svc.score(X_train, y_train) * 100, 2)
kfold = KFold(n_splits=5, random_state=22)
result_svm=cross_val_score(model,all_features,Targete,cv=10,scoring='accuracy')

print('The accuracy of the Support Vector Machines Classifier is',acc_linear_svc)
print('The cross validated score for Support Vector Machines Classifier is:',round(result_svm.mean()*100,2))

### 5. Random Forest:

In [ ]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, y_train)

Y_prediction = random_forest.predict(X_test)

random_forest.score(X_train, y_train)
acc_random_forest = round(random_forest.score(X_train, y_train) * 100, 2)

kfold = KFold(n_splits=10, random_state=22) # k=10, split the data into 10 equal parts
result_rm=cross_val_score(model,all_features,Targete,cv=10,scoring='accuracy')

print('The accuracy of the Random Forest Classifier is',acc_random_forest)
print('The cross validated score for Random Forest Classifier is:',round(result_rm.mean()*100,2))

### 6. Decision tree

In [ ]:
decision_tree = DecisionTreeClassifier() 
decision_tree.fit(X_train, y_train)
Y_pred = decision_tree.predict(X_test) 
acc_decision_tree = round(decision_tree.score(X_train, y_train) * 100, 2)

kfold = KFold(n_splits=10, random_state=22) # k=10, split the data into 10 equal parts
result_rm=cross_val_score(model,all_features,Targete,cv=10,scoring='accuracy')

print('The accuracy of the Random Forest Classifier is',acc_decision_tree)
print('The cross validated score for Random Forest Classifier is:',round(result_rm.mean()*100,2))

lets check which model performed best here.

## Which is the best Model ?

In [ ]:
results = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes',  
              'Decision Tree'],
    'Score': [acc_linear_svc, acc_knn, Log_acc, 
              acc_random_forest, nb_acc, acc_decision_tree]})
result_df = results.sort_values(by='Score', ascending=False)
result_df = result_df.set_index('Model')
result_df.head(9)

Here we can say random forest give best result here, so lets Understand little bit more about Random Forest.

### What is Random Forest ?
Random Forest is a supervised learning algorithm. Like you can already see from it’s name, it creates a forest and makes it somehow random. The „forest“ it builds, is an ensemble of Decision Trees, most of the time trained with the “bagging” method. The general idea of the bagging method is that a combination of learning models increases the overall result.

To say it in simple words: 

**Random forest builds multiple decision trees and merges them together to get a more accurate and stable prediction.**

One big advantage of random forest is, that it can be used for both classification and regression problems, which form the majority of current machine learning systems. With a few exceptions a random-forest classifier has all the hyperparameters of a decision-tree classifier and also all the hyperparameters of a bagging classifier, to control the ensemble itself.

The random-forest algorithm brings extra randomness into the model, when it is growing the trees. Instead of searching for the best feature while splitting a node, it searches for the best feature among a random subset of features. This process creates a wide diversity, which generally results in a better model. Therefore when you are growing a tree in random forest, only a random subset of the features is considered for splitting a node. You can even make trees more random, by using random thresholds on top of it, for each feature rather than searching for the best possible thresholds (like a normal decision tree does).

Below you can see how a random forest would look like with two trees:

<img src="https://i.ibb.co/TBbZnGC/two-tree-random-forest.png" alt="Random Forest" style="width:450px;height:300px;">
<ul>

<br>

we can start evaluating it’s performace in a more accurate way. Previously we only used accuracy.

The problem is just, that it’s more complicated to evaluate a classification model than a regression model. We will talk about this in the following section.
## Further Evaluation
----
### Confusion Matrix:

In [ ]:
predictions = cross_val_predict(random_forest, X_train, y_train, cv=3)
confusion_matrix(y_train, predictions)

The first row is about the not-survived-predictions: 331 passengers were **correctly classified as not survived (called true negatives)** and 50 where **wrongly classified as not survived (false positives)**.

The second row is about the survived-predictions: 79 passengers where **wrongly classified as survived (false negatives)** and 247 where **correctly classified as survived (true positives).**

### Precision and Recall:

In [ ]:
from sklearn.metrics import precision_score, recall_score
print("Precision:", precision_score(y_train, predictions))
print("Recall:",recall_score(y_train, predictions))

### F-Score
The F-score is computed with the harmonic mean of precision and recall. Note that it assigns much more weight to low values. As a result of that, the classifier will only get a high F-score, if both recall and precision are high.

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_train, predictions)

### ROC AUC Curve
Another way to evaluate and compare your binary classifier is provided by the ROC AUC Curve.

This curve plots the true positive rate (also called recall) against the false positive rate (ratio of incorrectly classified negative instances), instead of plotting the precision versus the recall.

In [ ]:
from sklearn.metrics import roc_curve
# compute true positive rate and false positive rate
y_scores = random_forest.predict_proba(X_train)
y_scores = y_scores[:,1]
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_train, y_scores)
# plotting them against each other
def plot_roc_curve(false_positive_rate, true_positive_rate, label=None):
    plt.plot(false_positive_rate, true_positive_rate, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'r', linewidth=4)
    plt.axis([-0.05, 1.05, -0.05, 1.05])
    plt.xlabel('False Positive Rate (FPR)', fontsize=16)
    plt.ylabel('True Positive Rate (TPR)', fontsize=16)

plt.figure(figsize=(14, 7))
plot_roc_curve(false_positive_rate, true_positive_rate)
plt.show()

## Conclusion
This was my very first Kaggle competition and climbing up the leaderboard one step at a time was definitely a really nice journey.

From the community, I learned that more feature engineering could be done here and for this reason, I will probably try a ticket grouping approach soon.

Thank you for taking the time to read through my first exploration of a Kaggle dataset.

For the moment, let me know if you found this notebook useful or you just liked it: I would really appreciate it!

## Plz Upvote!